In [2]:
import sys
!{sys.executable} -m pip install --upgrade pymorphy2

from pymorphy2 import MorphAnalyzer

parser = MorphAnalyzer()

In [3]:
import requests
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

# Загрузка файла
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)
rus_stops = response.text.splitlines()

punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)

parser = MorphAnalyzer()

def preprocess_gemsim(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text if word]  # Проверка на пустые строки
    return lemmatized_text


<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\]'
C:\Users\Сократ\AppData\Local\Temp\ipykernel_1232\1493062947.py:10: SyntaxWarning: invalid escape sequence '\]'
  punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'


In [4]:
import pandas as pd

# Открытие файла с явным указанием кодировки
with open("anek_utf8.txt", "r", encoding="utf-8") as file1:
    lines = []
    for line in file1:
        if line.strip():  # Проверка на пустую строку
            lines.append(line.strip())  # Удаление пробелов и символов новой строки

line_new = []
for i in range(len(lines) - 1):  # Используем 'i' вместо 'n'
    if lines[i][:15] == '<|startoftext|>':
        if lines[i + 1][:15] != '<|startoftext|>':
            line_new.append(lines[i][15:] + ' ' + lines[i + 1][15:])
        else:
            line_new.append(lines[i][15:])

# Создание DataFrame
ds_line = pd.DataFrame(line_new, columns=['text_base'])


In [5]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')#для табов

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
import nltk
nltk.download('punkt')

# ds_line["text_processed_new"] = ds_line["text_base"].apply(lambda text: preprocess_sklearn(text))
ds_line["text_processed"] = ds_line["text_base"].apply(lambda text: preprocess_gemsim(text))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
# Build the bigram  models
texts=(ds_line["text_processed"])
from gensim import corpora, models

bigram = models.Phrases(texts, min_count=3, threshold=5)
bigram_mod = models.phrases.Phraser(bigram)
texts_bi = make_bigrams(texts)
dictionary_bi = corpora.Dictionary(texts_bi)                 # составляем словарь из терминов с учетом биграмм слов предлжений
print('Размер словаря до фильтрации: {}'.format(len(dictionary_bi)))
dictionary_bi.filter_extremes(no_below=3, no_above=0.4, keep_n=3*10**6)
print('Размер словаря после фильтрации: {}'.format(len(dictionary_bi)))
corpus_bi = [dictionary_bi.doc2bow(text) for text in texts_bi]  # составляем корпус документов по словарю полученному с использованием биграмм

Размер словаря до фильтрации: 83955
Размер словаря после фильтрации: 37948


In [8]:
number_topics=20

In [9]:
def lda_model_gensim(corpus,dictionary,num_topics):
  ldamodel = models.ldamodel.LdaModel(
    corpus,
    id2word=dictionary,
    eval_every=20,
    num_topics=num_topics,
    passes=5
    )
  return ldamodel

In [10]:
ldamodel_bi=lda_model_gensim(corpus_bi,dictionary_bi,number_topics)

In [11]:
topics_gensim_bi = ldamodel_bi.show_topics(num_topics=number_topics, num_words=100,formatted=False)
topics_bi=pd.DataFrame(topics_gensim_bi)
topics_bi.columns=['topic num','topic_word']

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [13]:
vis_bi = gensimvis.prepare(ldamodel_bi, corpus_bi, dictionary_bi)

In [14]:
pyLDAvis.enable_notebook()

In [15]:
vis_bi

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.198495 -0.055501       1        1  7.419029
3      0.227961 -0.369767       2        1  6.502692
11    -0.173537 -0.027165       3        1  6.224027
7     -0.156983 -0.050877       4        1  5.844296
4     -0.027808  0.094818       5        1  5.322114
8     -0.147600 -0.062989       6        1  5.300212
18    -0.011967 -0.060730       7        1  5.053589
12    -0.075636  0.004214       8        1  5.026218
17    -0.030162  0.009890       9        1  4.924964
2      0.088240  0.134615      10        1  4.867922
19    -0.076821 -0.041031      11        1  4.749867
5      0.063152  0.064880      12        1  4.738320
10     0.022245  0.090709      13        1  4.550377
9     -0.014992  0.055131      14        1  4.440293
6      0.122828  0.078055      15        1  4.287637
0      0.139587  0.047001      16        1  4.252527
14    -0.027728 -0.035165      17        1  4.243329
13     0.061799 -0.053408      18        1  4.214218
16     0.067041  0.082311      19        1  4.100209
1      0.148876  0.095009      20        1  3.938161, topic_info=             Term          Freq         Total Category  logprob  loglift
24             ``  19424.000000  19424.000000  Default  30.0000  30.0000
23             ''  17673.000000  17673.000000  Default  29.0000  29.0000
320          жена   8983.000000   8983.000000  Default  28.0000  28.0000
27        женщина   6891.000000   6891.000000  Default  27.0000  27.0000
326           муж   5610.000000   5610.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
2625  волноваться    352.195979    353.486668  Topic20  -4.7694   3.2308
511        убрать    351.032125    352.322815  Topic20  -4.7727   3.2308
5918      детский    350.859112    352.149802  Topic20  -4.7732   3.2308
3909     являться    331.612236    332.902926  Topic20  -4.8296   3.2306
74            ...    700.362418  39595.113578  Topic20  -4.0819  -0.8004

[735 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
23        2  0.999946        ''
2441     15  0.997061     ''_.-
74        1  0.140724       ...
74        2  0.007223       ...
74        3  0.153883       ...
...     ...       ...       ...
777      12  0.997940     школа
1079     14  0.997999   экзамен
3909     20  0.997288  являться
1055      8  0.997357      язык
1640     20  0.997667      яйцо

[910 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 4, 12, 8, 5, 9, 19, 13, 18, 3, 20, 6, 11, 10, 7, 1, 15, 14, 17, 2])

In [16]:
from gensim.models.coherencemodel import CoherenceModel

# Вычисление коэренции тем
coherence_model_lda = CoherenceModel(model=ldamodel_bi, texts=texts_bi, dictionary=dictionary_bi, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print(f"Коэренция тем: {coherence_lda}")


Коэренция тем: 0.25226079588990463


In [18]:
from gensim.models.coherencemodel import CoherenceModel

# Перебор значений number_topics от 20 до 25
for number_topics in range(10, 15):
    # Обучение LDA модели
    ldamodel_bi = lda_model_gensim(corpus_bi, dictionary_bi, number_topics)
    
    # Вычисление коэренции тем
    coherence_model_lda = CoherenceModel(model=ldamodel_bi, texts=texts_bi, dictionary=dictionary_bi, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    print(f"Коэренция тем для {number_topics} тем: {coherence_lda}")


Коэренция тем для 10 тем: 0.32018511760838597
Коэренция тем для 11 тем: 0.28883372310396305
Коэренция тем для 12 тем: 0.2648323981592811
Коэренция тем для 13 тем: 0.28109587095050687
Коэренция тем для 14 тем: 0.26549515747061736


In [19]:
from gensim.models.coherencemodel import CoherenceModel

# Перебор значений number_topics от 20 до 25
for number_topics in range(15, 20):
    # Обучение LDA модели
    ldamodel_bi = lda_model_gensim(corpus_bi, dictionary_bi, number_topics)
    
    # Вычисление коэренции тем
    coherence_model_lda = CoherenceModel(model=ldamodel_bi, texts=texts_bi, dictionary=dictionary_bi, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    print(f"Коэренция тем для {number_topics} тем: {coherence_lda}")


Коэренция тем для 15 тем: 0.24228839705837224
Коэренция тем для 16 тем: 0.25744659283194515
Коэренция тем для 17 тем: 0.23362597735785826
Коэренция тем для 18 тем: 0.26474154356177526
Коэренция тем для 19 тем: 0.25238329653484093


In [ ]:
from gensim.models.coherencemodel import CoherenceModel

for number_topics in range(100, 105):
    # Обучение LDA модели
    ldamodel_bi = lda_model_gensim(corpus_bi, dictionary_bi, number_topics)
    
    # Вычисление коэренции тем
    coherence_model_lda = CoherenceModel(model=ldamodel_bi, texts=texts_bi, dictionary=dictionary_bi, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    print(f"Коэренция тем для {number_topics} тем: {coherence_lda}")


Коэренция тем для 100 тем: 0.42511817101507526
Коэренция тем для 101 тем: 0.4233209668067265
Коэренция тем для 102 тем: 0.41650490134076723
Коэренция тем для 103 тем: 0.41422200362429035
Коэренция тем для 104 тем: 0.4168880597622183


In [21]:
from gensim.models.coherencemodel import CoherenceModel

# Перебор значений number_topics от 20 до 25
n=500
delta=1
for number_topics in range(500, 501):
    # Обучение LDA модели
    ldamodel_bi = lda_model_gensim(corpus_bi, dictionary_bi, number_topics)
    
    # Вычисление коэренции тем
    coherence_model_lda = CoherenceModel(model=ldamodel_bi, texts=texts_bi, dictionary=dictionary_bi, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    print(f"Коэренция тем для {number_topics} тем: {coherence_lda}")


Коэренция тем для 500 тем: 0.47678365491755154


In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Перебор значений number_topics от 20 до 25
n=500
delta=1
for number_topics in range(n, n+delta):
    # Обучение LDA модели
    ldamodel_bi = lda_model_gensim(corpus_bi, dictionary_bi, number_topics)
    
    # Вычисление коэренции тем
    coherence_model_lda = CoherenceModel(model=ldamodel_bi, texts=texts_bi, dictionary=dictionary_bi, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    print(f"Коэренция тем для {number_topics} тем: {coherence_lda}")
